# Programming Assignment: Метрики качества классификации

## Введение

В задачах классификации может быть много особенностей, влияющих на подсчет качества: различные цены ошибок, несбалансированность классов и т.д. Из-за этого существует большое количество метрик качества — каждая из них рассчитана на определенное сочетание свойств задачи и требований к ее решению.

Меры качества классификации можно разбить на две большие группы: предназначенные для алгоритмов, выдающих номера классов, и для алгоритмов, выдающих оценки принадлежности к классам. К первой группе относятся доля правильных ответов, точность, полнота, F-мера. Ко второй — площади под `ROC`- или `PR`-кривой.

## Реализация в sklearn

Различные метрики качества реализованы в пакете `sklearn.metrics`. Конкретные функции указаны в инструкции по выполнению задания.

## Материалы

* [Подробнее о метриках качества](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture04-linclass.pdf)
* [Задачи по AUC-ROC](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/seminars/sem05-linclass.pdf)

## Инструкция по выполнению

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve

In [2]:
def write_answer(*ans, n):
    with open("ans{}.txt".format(n), "w") as f:
        answer = str(ans[0])
        for i in ans[1:]:
            answer += ' ' + str(i) 
        f.write(str(answer))

### Шаг 1:

Загрузите файл `classification.csv`. В нем записаны истинные классы объектов выборки (колонка `true`) и ответы некоторого классификатора (колонка `pred`).

In [3]:
df = pd.read_csv('classification.csv')
df.head()

,true,pred
0,1,0
1,1,1
2,1,1
3,0,0
4,1,1


### Шаг 2:

Заполните таблицу ошибок классификации:

Для этого подсчитайте величины `TP`, `FP`, `FN` и `TN` согласно их определениям. Например, `FP` — это количество объектов, имеющих класс 0, но отнесенных алгоритмом к классу 1. Ответ в данном вопросе — четыре числа через пробел.

In [4]:
TP = len(df[(df['true'] == 1) & (df['pred'] == 1)])
FP = len(df[(df['true'] == 0) & (df['pred'] == 1)])
FN = len(df[(df['true'] == 1) & (df['pred'] == 0)])
TN = len(df[(df['true'] == 0) & (df['pred'] == 0)])

print('TP:', TP, '\nFP:', FP, '\nFN:', FN, '\nTN:', TN)

write_answer(TP, FP, FN, TN, n = 1)

TP: 43 
FP: 34 
FN: 59 
TN: 64


### Шаг 3: 

Посчитайте основные метрики качества классификатора:

* `Accuracy` (доля верно угаданных) — `sklearn.metrics.accuracy_score`
* `Precision` (точность) — `sklearn.metrics.precision_score`
* `Recall` (полнота) — `sklearn.metrics.recall_score`
* `F-мера` — `sklearn.metrics.f1_score`

В качестве ответа укажите эти четыре числа через пробел.

In [5]:
a =  accuracy_score(df['true'], df['pred'])
p = precision_score(df['true'], df['pred'])
r = recall_score(df['true'], df['pred'])
f1 = f1_score(df['true'], df['pred'])
print('Accuracy:', a, '\nPrecision:', p, '\nRecall:', r, '\nF measure:', f1)

write_answer(a, p, r, f1, n = 2)

Accuracy: 0.535 
Precision: 0.5584415584415584 
Recall: 0.4215686274509804 
F measure: 0.48044692737430167


### Шаг 4:

Имеется четыре обученных классификатора. В файле `scores.csv` записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:

* для логистической регрессии — вероятность положительного класса (колонка `score_logreg`),
* для `SVM` — отступ от разделяющей поверхности (колонка `score_svm`),
* для метрического алгоритма — взвешенная сумма классов соседей (колонка `score_knn`),
* для решающего дерева — доля положительных объектов в листе (колонка `score_tree`).

Загрузите этот файл.

In [6]:
df_scores = pd.read_csv('scores.csv')
df_scores.head()

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263


### Шаг 5:

Посчитайте площадь под `ROC`-кривой для каждого классификатора. Какой классификатор имеет наибольшее значение метрики `AUC-ROC` (укажите название столбца)? Воспользуйтесь функцией `sklearn.metrics.roc_auc_score`.

In [7]:
roc_auc_scores = {col: roc_auc_score(df_scores['true'], df_scores[col]) for col in df_scores.columns[1:]}
best_clf = max(roc_auc_scores, key=roc_auc_scores.get) 

print('ROC-AUC scores:', roc_auc_scores)
print('Classifier with max score:', best_clf)

write_answer(best_clf, n = 3)   

ROC-AUC scores: {'score_logreg': 0.719187675070028, 'score_svm': 0.7086834733893557, 'score_knn': 0.6351540616246498, 'score_tree': 0.6919267707082833}
Classifier with max score: score_logreg


### Шаг 6: 

Какой классификатор достигает наибольшей точности (`Precision`) при полноте (`Recall`) не менее 70% ?

Чтобы получить ответ на этот вопрос, найдите все точки `precision-recall`-кривой с помощью функции `sklearn.metrics.precision_recall_curve`. Она возвращает три массива: `precision`, `recall`, `thresholds`. В них записаны точность и полнота при определенных порогах, указанных в массиве thresholds. Найдите максимальной значение точности среди тех записей, для которых полнота не меньше, чем 0.7.

In [8]:
pr = {}

for col in df_scores.columns[1:]:
    precision, recall, thresholds = precision_recall_curve(df_scores['true'], df_scores[col])
    b = np.where(recall>=0.7)
    pr[col] = max(precision[list(b)])

best_clf = max(pr, key=pr.get) 

print('Precision scores:', pr)
print('Classifier with max precision:', best_clf)

write_answer(best_clf, n = 4)

Precision scores: {'score_logreg': 0.6302521008403361, 'score_svm': 0.6228070175438597, 'score_knn': 0.6065573770491803, 'score_tree': 0.6517857142857143}
Classifier with max precision: score_tree


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.42. При необходимости округляйте дробную часть до двух знаков.